# Homework 5: Part I

In [1]:
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))

with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 87 seconds.


In [6]:
import pymc as pm
import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv('Salary_dataset.csv')

# Extract the features (X) and target variable (y)
X = data['YearsExperience'].values.reshape(-1, 1)
y = data['Salary'].values.reshape(-1, 1)

n, p = X.shape

# Bayesian Linear Regression Model
with pm.Model() as MLR:
    # Priors
    betas = pm.MvNormal('betas', mu=np.zeros((p, 1)), cov=np.eye(p), shape=(p, 1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0)  # Half normal prior for sigma
    
    # Likelihood
    y_observed = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

# Sampling from the model
with MLR:
    idata = pm.sample()


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4 seconds.


# Homework 5: Part II

1. 
\begin{align*}
p(\beta | \sigma^2, X, y) &= \mathcal{N}(\beta | \bar{\beta}, \text{Var}[\beta]) \\
&= \mathcal{N}(\beta | (X^TX)^{-1}X^Ty, \sigma^2(X^TX)^{-1})
\end{align*}


2. 
\begin{align*}
\mathbb{E}[\beta | \Sigma, X, y] &= (X^T \Sigma^{-1} X)^{-1} X^T \Sigma^{-1} y
\end{align*}


3. 
\begin{align*}
\text{For } \mathbb{E}[\beta | \Sigma, X, y] \text{ to equal } (X^T X)^{-1} X^T y, \text{ which is the ordinary least squares estimate, we would need the prior variance of } \Sigma \beta \text{ to go to infinity (or equivalently, the precision to go to zero), meaning that the prior has no effect on the posterior distribution.}
\end{align*}



4. 
\begin{align*}
&\text{This is the expected value of the predicted values } \hat{y} \text{ given the model parameters. To make the expected value of the predictions equal to the projections of } y \text{ onto the column space of } X, \\
&\text{we would again require that the prior has no influence on the posterior, which would again imply an infinite prior variance for } \beta.
\end{align*}



5. 
\begin{align*}
\text{Var} [\beta | \Sigma, X, y] &= \begin{aligned}[t]
& (X^T \Sigma^{-1} X)^{-1} \\
& + \Sigma \beta ^{-1}
\end{aligned}
\end{align*}
This equation shows how the variance of $\beta$ is influenced by both the design matrix $X$ and the covariance matrix $\Sigma$, as well as the prior variance $\Sigma \beta $.



# Homework 5: Part III

In [6]:
import pymc as pm
import numpy as np
from scipy import stats

p = 10  # Dimensionality of the multivariate normal
Psi = np.eye(p)  # Scale matrix for the Wishart distribution
a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)  # Inverse-Wishart sample for the covariance matrix

n = 1000  # Number of data points
y = stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)  # Simulated data points

# Using PyMC3 to define the model
with pm.Model() as MNV_LKJ:
    # Define the Cholesky factor of the covariance matrix
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=p), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    Sigma = pm.Deterministic('Sigma', L.dot(L.T))  # Reconstructing the covariance matrix

    # Define the prior for the mean vector using a small constant for the covariance for numerical stability
    mu = pm.MvNormal('mu', mu=np.zeros(p), cov=np.eye(p)*1e-6, shape=p)

    # The observed data likelihood, parameterized using the Cholesky factor
    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

# Sampling from the model
with MNV_LKJ:
    trace = pm.sample(1000)  

# The trace object now contains the samples for the posterior distribution


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L, mu]


/opt/conda/lib/python3.11/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 371 seconds.
